# Fine-tuning protein language models

**Q6.** Describe the problem of predicting the subcellular location of (prokaryotic) proteins as described in [Moreno et al., 2024](https://doi.org/10.1093/bioinformatics/btae677)? Think of a biological question one could answer with proteome-wide predictions of subcellular location, and potential follow-up experiments?

In [1]:
import numpy as np, pandas as pd, sklearn.preprocessing
import datasets, evaluate, transformers # Hugging Face libraries https://doi.org/10.18653/v1/2020.emnlp-demos.6
import Bio.SeqIO.FastaIO # Biopython for reading fasta files
from IPython.display import display, HTML
random_number = 4 # https://xkcd.com/221/

In [2]:
# Uncomment & execute once to download data from https://services.healthtech.dtu.dk/services/DeepLocPro-1.0/
#!mkdir -p data
#!curl https://services.healthtech.dtu.dk/services/DeepLocPro-1.0/data/graphpart_set.fasta -o data/graphpart_set.fasta

**Q7.** Look at the contents of `df_data`, how was the column `fold_id` defined in the paper? What exact set of sequences are in this data set (check number of rows)? 

In [3]:
with open('data/graphpart_set.fasta') as handle:
    fasta_cols = ['header', 'sequence']
    df_data = pd.DataFrame.from_records([values for values in Bio.SeqIO.FastaIO.SimpleFastaParser(handle)], columns=fasta_cols)
header_cols = ['uniprot_id', 'subcellular_location', 'organism_group', 'fold_id']
df_data[header_cols] = df_data['header'].str.split('|', expand=True)
final_cols = ['uniprot_id']
df_data = df_data[['uniprot_id', 'subcellular_location', 'organism_group', 'fold_id', 'sequence']].astype({'fold_id': int}).sort_values('fold_id')
# The data set has 11 906 items; paper assembles 11 970 sequences, but has to remove 64 proteins to avoid homologous sequences between different folds (GraphPart)
df_data

,uniprot_id,subcellular_location,organism_group,fold_id,sequence
8560,Q8A0Z3,Cytoplasmic,negative,0,MAVTMADITKLRKMTGAGMMDCKNALTEAEGDYDKAMEIIRKKGQA...
8568,Q8A2N8,Cytoplasmic,negative,0,MIMSKETLIKSIREIPDFPIPGILFYDVTTLFKDPWCLQELSNIMF...
2593,P32709,CYtoplasmicMembrane,negative,0,MTQTSAFHFESLVWDWPIAIYLFLIGISAGLVTLAVLLRRFYPQAG...
61,E7FHF8,Cytoplasmic,archaea,0,MKLGVFELTDCGGCALNLLFLYDKLLDLLEFYEIAEFHMATSKKSR...
63,E7FHU4,Cytoplasmic,archaea,0,MGKVRIGFYALTSCYGCQLQLAMMDELLQLIPNAEIVCWFMIDRDS...
...,...,...,...,...,...
5239,Q97F85,Cytoplasmic,positive,4,MRKLFTSESVTEGHPDKICDQISDAILDAILEKDPNGRVACETTVT...
5226,P33656,Cytoplasmic,positive,4,MKNKTEVKNGGEKKNSKKVSKEESAKEKNEKMKIVKNLIDKGKKSG...
11894,P13949,OuterMembrane,negative,4,MCALDRRERPLNSQSVNKYILNVQNIYRNSPVPVCVRNKNRKILYA...
11890,P42185,Extracellular,negative,4,MRLRFSVPLFFFGCVFVHGVFAGPFPPPGMSLPEYWGEEHVWWDGR...


In [4]:
# Encode subcellular location as numerical labels
subcellular_location_encoder = sklearn.preprocessing.LabelEncoder()
subcellular_location_encoder.fit(df_data['subcellular_location'])
df_data['label'] = subcellular_location_encoder.transform(df_data['subcellular_location'])
df_data

,uniprot_id,subcellular_location,organism_group,fold_id,sequence,label
8560,Q8A0Z3,Cytoplasmic,negative,0,MAVTMADITKLRKMTGAGMMDCKNALTEAEGDYDKAMEIIRKKGQA...,2
8568,Q8A2N8,Cytoplasmic,negative,0,MIMSKETLIKSIREIPDFPIPGILFYDVTTLFKDPWCLQELSNIMF...,2
2593,P32709,CYtoplasmicMembrane,negative,0,MTQTSAFHFESLVWDWPIAIYLFLIGISAGLVTLAVLLRRFYPQAG...,0
61,E7FHF8,Cytoplasmic,archaea,0,MKLGVFELTDCGGCALNLLFLYDKLLDLLEFYEIAEFHMATSKKSR...,2
63,E7FHU4,Cytoplasmic,archaea,0,MGKVRIGFYALTSCYGCQLQLAMMDELLQLIPNAEIVCWFMIDRDS...,2
...,...,...,...,...,...,...
5239,Q97F85,Cytoplasmic,positive,4,MRKLFTSESVTEGHPDKICDQISDAILDAILEKDPNGRVACETTVT...,2
5226,P33656,Cytoplasmic,positive,4,MKNKTEVKNGGEKKNSKKVSKEESAKEKNEKMKIVKNLIDKGKKSG...,2
11894,P13949,OuterMembrane,negative,4,MCALDRRERPLNSQSVNKYILNVQNIYRNSPVPVCVRNKNRKILYA...,4
11890,P42185,Extracellular,negative,4,MRLRFSVPLFFFGCVFVHGVFAGPFPPPGMSLPEYWGEEHVWWDGR...,3


**Q8.** How were the data partitioned during training and evaluation in the paper? What does the code below do, and how does it compare to the approach taken in the paper?

In [5]:
train_id = {0, 1, 2}
eval_id = {3}
test_id = {4}

df_train = df_data.query('fold_id in @train_id')#.groupby('subcellular_location').sample(n=50, random_state=random_number)
df_eval = df_data.query('fold_id in @eval_id')
df_test = df_data.query('fold_id in @test_id')
print(len(df_train), 'records in training data:')
print(df_train['subcellular_location'].value_counts())
print()
print(len(df_eval), 'records in eval data:')
print(df_eval['subcellular_location'].value_counts())
print()
print(len(df_test), 'records in test data:')
print(df_test['subcellular_location'].value_counts())

7336 records in training data:
subcellular_location
Cytoplasmic            4345
CYtoplasmicMembrane    1534
Extracellular           645
OuterMembrane           433
Periplasmic             320
Cellwall                 59
Name: count, dtype: int64

2579 records in eval data:
subcellular_location
Cytoplasmic            1568
CYtoplasmicMembrane     476
Extracellular           224
OuterMembrane           159
Periplasmic             136
Cellwall                 16
Name: count, dtype: int64

1991 records in test data:
subcellular_location
Cytoplasmic            972
CYtoplasmicMembrane    525
Extracellular          208
OuterMembrane          164
Periplasmic            110
Cellwall                12
Name: count, dtype: int64


In [6]:
# Prepare train/eval/test data sets for ESM2 model
model_checkpoint = 'facebook/esm2_t6_8M_UR50D'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoint)

train_tokenized = tokenizer(df_train['sequence'].tolist(), truncation=True, max_length=1024)
eval_tokenized = tokenizer(df_eval['sequence'].tolist(), truncation=True, max_length=1024)
test_tokenized = tokenizer(df_test['sequence'].tolist(), truncation=True, max_length=1024)

train_dataset = datasets.Dataset.from_dict(train_tokenized).add_column('labels', df_train['label'].tolist())
eval_dataset = datasets.Dataset.from_dict(eval_tokenized).add_column('labels', df_eval['label'].tolist())
test_dataset = datasets.Dataset.from_dict(test_tokenized).add_column('labels', df_test['label'].tolist())

**Q9.** There's a warning about uninitialized weights when loading the ESM-2 model using `AutoModelForSequenceClassification` below. Describe the part of the network that has the uninitialized weights. How does the new part connect to the rest?

In [7]:
# Uninitialised weights are in EsmClassificationHead which implements feed-forward network on the embeddings of the start-of-sequence/CLS token
# https://github.com/huggingface/transformers/blob/main/src/transformers/models/esm/modeling_esm.py#L1239-L1246 
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=df_data['label'].nunique())
model

Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


EsmForSequenceClassification(
  (esm): EsmModel(
    (embeddings): EsmEmbeddings(
      (word_embeddings): Embedding(33, 320, padding_idx=1)
      (dropout): Dropout(p=0.0, inplace=False)
      (position_embeddings): Embedding(1026, 320, padding_idx=1)
    )
    (encoder): EsmEncoder(
      (layer): ModuleList(
        (0-5): 6 x EsmLayer(
          (attention): EsmAttention(
            (self): EsmSelfAttention(
              (query): Linear(in_features=320, out_features=320, bias=True)
              (key): Linear(in_features=320, out_features=320, bias=True)
              (value): Linear(in_features=320, out_features=320, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
              (rotary_embeddings): RotaryEmbedding()
            )
            (output): EsmSelfOutput(
              (dense): Linear(in_features=320, out_features=320, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (LayerNorm): LayerNorm((320,), eps=1e-05,

In [8]:
# Track accuracy and macro F1 score throughout the training
# https://huggingface.co/docs/transformers/en/training#evaluate
metric_accuracy = evaluate.load('accuracy')
metric_f1 = evaluate.load('f1')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return {
        'accuracy': metric_accuracy.compute(predictions=predictions, references=labels)['accuracy'],
        'f1_macro': metric_f1.compute(predictions=predictions, references=labels, average='macro')['f1'],
    }

**Q10.** The fine-tuning may fail by running out of GPU memory. Look up `per_device_train_batch_size` and `per_device_eval_batch_size` in the [TrainingArguments docs](https://huggingface.co/docs/transformers/v4.49.0/en/main_classes/trainer#transformers.TrainingArguments). How would you adjust these parameters to use less GPU memory?

In [9]:
# Set up fine-tuning
trainer_args = transformers.TrainingArguments(
    output_dir=f'{model_checkpoint}-subcellular_location',
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    # Reduce from default (8) to 4 to run on MOL GPUs
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
)

trainer = transformers.Trainer(
    model,
    trainer_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

retrained = trainer.train()

/scratch/tmp.24154812.jjaenes/ipykernel_2512137/2369812818.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = transformers.Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.469600,0.540398,0.863901,0.614198
2,0.310300,0.513992,0.882513,0.657244
3,0.232600,0.482931,0.897635,0.770760


In [10]:
# Use fine-tuned model to predict on held-out test data
retrained_predict = trainer.predict(test_dataset=test_dataset)
retrained_predict.metrics

{'test_loss': 0.6614657640457153,
 'test_accuracy': 0.8709191361125063,
 'test_f1_macro': 0.7136070826015466,
 'test_runtime': 5.2415,
 'test_samples_per_second': 379.851,
 'test_steps_per_second': 95.011}

In [11]:
# Convert probabilities into discrete predictions by taking the max probability
test_labels = np.argmax(retrained_predict.predictions, axis=-1)
# Sanity-check by manualy calculating the accuracy
print(sum(test_labels == test_dataset['labels']) / len(test_dataset))
test_labels

0.8709191361125063


array([0, 2, 4, ..., 2, 5, 4])

**Q11.** Adjust the code below to re-produce the cross-validation results as in Table 2 of the paper. Fine-tune+test separately on every fold, and gather the results in `predicted_labels`.

In [12]:
fold_id = set(df_data.fold_id)
predicted_labels = []
for test_id in sorted(fold_id):
    eval_id = (test_id + 1) % 5
    train_id = fold_id - set([eval_id, test_id])

    df_train = df_data.query('fold_id in @train_id')#.groupby('subcellular_location').sample(n=10, random_state=random_number)
    df_eval = df_data.query('fold_id == @eval_id')
    df_test = df_data.query('fold_id == @test_id')
    print(train_id, eval_id, test_id, len(df_train), len(df_eval), len(df_test))

    # Re-train model using train/eval/test as defined abobe
    model = transformers.AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=df_data['label'].nunique())

    # Prepare train/eval/test data sets for ESM2 model
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoint)

    train_tokenized = tokenizer(df_train['sequence'].tolist(), truncation=True, max_length=1024)
    eval_tokenized = tokenizer(df_eval['sequence'].tolist(), truncation=True, max_length=1024)
    test_tokenized = tokenizer(df_test['sequence'].tolist(), truncation=True, max_length=1024)

    train_dataset = datasets.Dataset.from_dict(train_tokenized).add_column('labels', df_train['label'].tolist())
    eval_dataset = datasets.Dataset.from_dict(eval_tokenized).add_column('labels', df_eval['label'].tolist())
    test_dataset = datasets.Dataset.from_dict(test_tokenized).add_column('labels', df_test['label'].tolist())

    # Set up fine-tuning
    trainer_args = transformers.TrainingArguments(
        output_dir=f'{model_checkpoint}-subcellular_location-{test_id}',
        eval_strategy='epoch',
        save_strategy='epoch',
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
        # Adjust if needed
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
    )

    trainer = transformers.Trainer(
        model,
        trainer_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    retrained = trainer.train()

    # Predict labels, and gather the predictions for the held-out test data into predicted_labels
    retrained_predict = trainer.predict(test_dataset=test_dataset)
    predicted_labels += list(np.argmax(retrained_predict.predictions, axis=-1))

{2, 3, 4} 1 0 6668 2684 2554


Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/scratch/tmp.24154812.jjaenes/ipykernel_2512137/516577029.py:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = transformers.Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.476100,0.399070,0.904993,0.671837
2,0.375300,0.475272,0.898286,0.667752
3,0.231500,0.460753,0.904247,0.674040


Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{0, 3, 4} 2 1 7124 2098 2684


...

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.483500,0.633639,0.843295,0.633120
2,0.316100,0.634640,0.864390,0.711928
3,0.221500,0.646642,0.872426,0.738894


Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{1, 2, 3} 0 4 7361 2554 1991


/scratch/tmp.24154812.jjaenes/ipykernel_2512137/516577029.py:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = transformers.Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.485800,0.448124,0.883712,0.638944
2,0.313200,0.513850,0.892326,0.686455
3,0.215500,0.500837,0.898982,0.733925


**Q12.** Discuss differences between the methodology and the resulting performance of the approach taken in the paper, and the reproduction.

In [13]:
# Show table with performance metrics split by organism to match Table 2
def calculate_stats_(df):
    accuracy = metric_accuracy.compute(predictions=df.predicted_labels.values, references=df.label.values)['accuracy']
    f1_macro = metric_f1.compute(predictions=df.predicted_labels.values, references=df.label.values, average='macro')['f1']
    return pd.Series({
        'size': '{:d}'.format(len(df)),
        'accuracy': '{:.2f}'.format(accuracy),
        'f1_macro': '{:.2f}'.format(f1_macro),
    })

# Paper vs reproduction:
# 650M vs 8M parameters
# 60 vs 3 epochs
# optimise hyperparameters (learning rate, batch size, dropout rate) vs use defaults
# ??? vs 30 min training time
# 0.92 vs 0.89 accuracy (overall)
# 0.80 vs 0.73 F1 score (overall)
df_data['predicted_labels'] = predicted_labels
pd.concat([
    calculate_stats_(df_data),
    calculate_stats_(df_data.query('organism_group == "archaea"')),
    calculate_stats_(df_data.query('organism_group == "positive"')),
    calculate_stats_(df_data.query('organism_group == "negative"')),
], axis=1).set_axis(['Overall', 'Archaea' , 'Gram pos', 'Gram neg'], axis=1)

,Overall,Archaea,Gram pos,Gram neg
size,11906,283,3206,8417
accuracy,0.89,0.84,0.92,0.89
f1_macro,0.71,0.43,0.48,0.70
